In [1]:
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import os
import gc
import joblib
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
#Load the data and make new df
train = pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")
train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [3]:
def complete_col(train,test,col):
    temp = []
    for i in train[col]:
        if not i in temp:
            temp.append(i)
    for i in test[col]:
        if not i in temp:
            temp.append(i)
    return temp

In [4]:
def label(train,test,col,new_train):
    lbl_enc = preprocessing.LabelEncoder()
    lbl_enc.fit(complete_col(train,test,col))
    new_train[col] = lbl_enc.transform(train[col]).astype('int64')
        
def one_hot(train, test ,col, new_train):
    encoder = OneHotEncoder()
    encoder.fit([[i] for i in complete_col(train,test,col)])
    train_data = np.array([[i] for i in train[col]])
    enc = encoder.transform(train_data).toarray()
    new_train[col] = np.array(enc[:,0]).astype('int32')


In [9]:
def make_df(start, num_dfs):
    encodings = [label, one_hot]
    safe_encodings = [label]
    for df in tqdm(range(num_dfs)):
        temp_df = pd.DataFrame()
        temp_df_test = pd.DataFrame()
        for col in train.columns[1:len(train.columns)-1]:
            if test[col].nunique() > 100:
                f = np.random.choice(safe_encodings)
            else:
                f = np.random.choice(encodings)
            f(train, test, col, temp_df)
            f(test, test, col, temp_df_test)
        temp_df.to_csv('encodings/' + str(start+df))
        temp_df_test.to_csv('encodings/test' + str(start+df))
        temp_df = None

In [10]:
make_df(0,50)


100%|██████████| 50/50 [1:24:19<00:00, 93.83s/it]